# Crawling BTS Lyrics

In [5]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import json
import csv
import re
from time import sleep

In [6]:
# use google chrome incognito mode for scraping
option = webdriver.ChromeOptions()
option.add_argument(" - incognito")

header = {'User-Agent': ''}

In [7]:
# open using chromeDriver
browser = webdriver.Chrome(executable_path='/Library/Application Support/Google/chromedriver', chrome_options=option)
browser.implicitly_wait(3)

# This is the link to BTS's songs page on Melon
browser.get("https://www.melon.com/artist/song.htm?artistId=672375")

In [8]:
# The average xpath for each lyric element looks something like this

# //*[@id="frm"]/div/table/tbody/tr[1]/td[3]/div/div/a[1]/span
# //*[@id="frm"]/div/table/tbody/tr[2]/td[3]/div/div/a[1]/span

for page in range(1, 6):
    result = list()
    sleep(3)
    for i in range(0, 50):
        song_selector = '//*[@id="frm"]/div/table/tbody/tr[' + str(i + 1) +']/td[3]/div/div/a[1]'

        try:
            song_item = browser.find_element_by_xpath(song_selector)
        except:
            print('end of songs')
            break;

        #clean values and get unique song_id
        song_id = re.sub('[^0-9]', '', song_item.get_attribute("href"))

        # print(song_id)

        req = requests.get('http://www.melon.com/song/lyrics.htm?songId=' + song_id, headers = header)
        html = req.text
        soup = BeautifulSoup(html, "html.parser")
        
        title = soup.find(attrs={"class": "song_name"}).text.replace('곡명', '').strip()

        # collect genre to later filter japanese lyrics etc.
        genre = soup.select('#downloadfrm > div > div > div.entry > div.meta > dl > dd')[2].text

        if soup.find_all(attrs={"class": "lyric"}):
            lyric = re.sub('<[^>]*>|\s|\[|\]', ' ', str(soup.find_all(attrs={"class": "lyric"})[0]))
            lyric = re.sub('^\s*|\s+$', '', lyric)
        else: # print blank if no lyrics 
            lyric = ''

        result.append({
            'title': title,
            'genre': genre,
            'lyric': lyric
        })

    with open('./btslyrics.json', 'a+', encoding='utf-8') as file:
        j = json.dumps(result, ensure_ascii=False)
        file.write(j)
            
    button_selector = '//*[@id="pageObjNavgation"]/div/span/a['+ str(page) +']'
    button = browser.find_element_by_xpath(button_selector)
    button.click()